# Bibliotecas e leitura dos dados

In [1]:
import preprocessing as pp
import load_files as lf
import nlp_algorithms as nlp

import pandas as pd
import numpy as np
import time

from sentence_transformers import SentenceTransformer

c:\Users\lucas\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_colwidth', -1)

train_v1 = pd.read_csv('dados/train v0.1.csv')

train_v1_enen = train_v1[(train_v1['url1_lang'] == 'en') & (train_v1['url2_lang'] == 'en')]

train_v1_enen.head()

C:\Users\lucas\AppData\Local\Temp\ipykernel_20500\4133509158.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,https://web.archive.org/web/www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://web.archive.org/web/www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,https://web.archive.org/web/www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://web.archive.org/web/www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,https://web.archive.org/web/gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://web.archive.org/web/gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,https://web.archive.org/web/news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://web.archive.org/web/www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [3]:
test = pd.read_csv('dados/final_evaluation_data.csv')

test_enen = test[(test['url1_lang'] == 'en') & (test['url2_lang'] == 'en')]

test_enen.head()

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-men-stole-tools-from-lowes-in-davie/,https://wsvn.com/news/local/no-swim-advisory-lifted-for-deerfield-beach-pier/,https://web.archive.org/web/https://wsvn.com/news/local/broward/police-2-men-stole-tools-from-lowes-in-davie/,https://web.archive.org/web/https://wsvn.com/news/local/no-swim-advisory-lifted-for-deerfield-beach-pier/,1.5,4.0,2.0,4.0,3.5,1.0,1.500000
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-database-leaked-customer-government-and-military-personal-records/,https://securityboulevard.com/2019/10/best-westerns-massive-data-leak-179gb-amazon-database-open-to-all/,https://web.archive.org/web/https://www.zdnet.com/article/autoclerk-database-leaked-customer-government-and-military-personal-records/,https://web.archive.org/web/https://securityboulevard.com/2019/10/best-westerns-massive-data-leak-179gb-amazon-database-open-to-all/,1.0,2.0,1.0,1.0,1.0,3.5,2.500000
2,en,en,1484039488_1484261803,https://www.presstelegram.com/2019/12/31/ducks-are-own-worst-enemies-in-sloppy-loss-in-las-vegas/,https://boingboing.net/2020/01/01/woody-guthries-1943-new-year.html,https://web.archive.org/web/https://www.presstelegram.com/2019/12/31/ducks-are-own-worst-enemies-in-sloppy-loss-in-las-vegas/,https://web.archive.org/web/https://boingboing.net/2020/01/01/woody-guthries-1943-new-year.html,5.0,4.0,3.0,4.0,4.0,4.0,3.666667
3,en,en,1484332324_1484796748,https://www.financialexpress.com/india-news/west-bengal-republic-day-tableau-rejected-by-centre-mamata-banerjee-latest-news/1810097/,https://www.news18.com/news/india/not-the-first-congress-did-7-times-bjp-reminds-as-row-over-bengal-maharashtra-r-day-tableau-continues-2443643.html,https://web.archive.org/web/https://www.financialexpress.com/india-news/west-bengal-republic-day-tableau-rejected-by-centre-mamata-banerjee-latest-news/1810097/,https://web.archive.org/web/https://www.news18.com/news/india/not-the-first-congress-did-7-times-bjp-reminds-as-row-over-bengal-maharashtra-r-day-tableau-continues-2443643.html,1.0,1.5,1.0,1.5,1.5,1.5,2.000000
4,en,en,1484012256_1484419682,https://www.birminghammail.co.uk/whats-on/music-nightlife-news/bars-clubs-you-loved-lost-17492049,http://m.fightbacknews.org/2020/1/1/top-20-films-2010s,https://web.archive.org/web/https://www.birminghammail.co.uk/whats-on/music-nightlife-news/bars-clubs-you-loved-lost-17492049,https://web.archive.org/web/http://m.fightbacknews.org/2020/1/1/top-20-films-2010s,4.0,4.0,1.0,2.5,4.0,2.5,2.500000


In [4]:
train_v1_enen.shape

(1761, 14)

In [5]:
test_enen.shape

(236, 14)

## Leitura das URLs baixadas para cada dataset (train e test)

In [6]:
data_path = 'dados/train v0.1/'

lista_docs = []
lista_error = []
lista_vazio = []
values = train_v1_enen[['pair_id', 'Overall']]

for index, values in values.iterrows():
    
    try:
        
        json_pair = lf.get_json_document_pair(data_path, values['pair_id'])
        text_doc1 = json_pair[0]['text']
        text_doc2 = json_pair[1]['text']
        
        if ( len(text_doc1) > 0 ) and ( len(text_doc2) > 0 ):
            lista_docs.append((values['pair_id'], text_doc1, text_doc2, values['Overall']))
        else:
            lista_vazio.append(values['pair_id'])
    
    except:
        lista_error.append(values['pair_id'])

In [7]:
df_text = pd.DataFrame(lista_docs,  columns=['pair_id', 'doc1', 'doc2', 'Overall'])

In [8]:
df_text.shape

(1357, 4)

In [9]:
data_path = 'dados/output_dir_test_enen/'

lista_docs = []
lista_error = []
lista_vazio = []
values = test_enen[['pair_id', 'Overall']]

for index, values in values.iterrows():
    
    try:
        
        json_pair = lf.get_json_document_pair(data_path, values['pair_id'])
        text_doc1 = json_pair[0]['text']
        text_doc2 = json_pair[1]['text']
        
        if ( len(text_doc1) > 0 ) and ( len(text_doc2) > 0 ):
            lista_docs.append((values['pair_id'], text_doc1, text_doc2, values['Overall']))
        else:
            lista_vazio.append(values['pair_id'])
    
    except:
        lista_error.append(values['pair_id'])

In [10]:
df_text_test = pd.DataFrame(lista_docs,  columns=['pair_id', 'doc1', 'doc2', 'Overall'])

In [11]:
df_text_test.shape

(235, 4)

## Pre-processamento

In [12]:
'''pre_processing_list = [
    {"tokenization": True},
    {"no_url": True, "basic_processing": True, "tokenization": True}
    ]'''

pre_processing_list = [
    {"tokenization": True}
    ]

df_pp = pp.pre_process_all(df_text, pre_processing_list)
df_pp_test = pp.pre_process_all(df_text_test, pre_processing_list)

# Algoritmos

In [13]:
df_results = df_pp_test[["pair_id", "Overall"]]
df_results.head(1)

,pair_id,Overall
0,1484189203_1484121193,3.5


## BERT

In [14]:
pre_processing_list

[{'tokenization': True}]

In [15]:
model_name = 'all-MiniLM-L12-v2'
#model_name = 'multi-qa-mpnet-base-dot-v1'
#model_name = 'all-distilroberta-v1'
#model_name = 'all-MiniLM-L12-v2'
#model_name = 'multi-qa-distilbert-cos-v1'
#model_name = 'all-MiniLM-L6-v2'

model = SentenceTransformer('sentence-transformers/'+model_name)

(df_bert, time_bert) = nlp.apply_bert(df_pp[:50], df_pp_test, len(pre_processing_list), model, model_name, fine_tune=True)

df_results = df_results.join(df_bert)

Epoch: 100%|██████████| 2/2 [00:26<00:00, 13.41s/it]


In [16]:
df_bert.to_csv('./resultados/'+model_name+'.csv', index = False)
time_bert.to_csv('./resultados/'+model_name+'_time.csv', index = False)

In [17]:
df_results.corr()

,Overall,bert_all-MiniLM-L12-v20
Overall,1.000000,-0.719582
bert_all-MiniLM-L12-v20,-0.719582,1.000000


## USE (Universal Sentence Encoder)

In [18]:
'''pre_processing_list = [
    {},
    {"no_url": True, "basic_processing": True}
    ]

df_pp = pp.pre_process_all(df_text, pre_processing_list)
df_pp_test = pp.pre_process_all(df_text_test, pre_processing_list)'''

'pre_processing_list = [\n    {},\n    {"no_url": True, "basic_processing": True}\n    ]\n\ndf_pp = pp.pre_process_all(df_text, pre_processing_list)\ndf_pp_test = pp.pre_process_all(df_text_test, pre_processing_list)'

In [19]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re
import math
import scipy

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [20]:
doc1_name = 'doc1'
doc2_name = 'doc2'

sts_data = df_pp_test[['Overall', doc1_name, doc2_name]]#sts_dev

def run_sts_benchmark(batch):
  start_time = time.time()
  sts_encode1 = tf.nn.l2_normalize(embed(tf.constant(batch[doc1_name].tolist())), axis=1)
  sts_encode2 = tf.nn.l2_normalize(embed(tf.constant(batch[doc2_name].tolist())), axis=1)

  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
  """Returns the similarity scores"""
  runtime = time.time()-start_time
  return (scores, runtime)

dev_scores = sts_data['Overall'].tolist()
scores = []
#for batch in np.array_split(sts_data, 10):
#  (df_use, time_use) = run_sts_benchmark(batch)
#  scores.extend(df_use)

(df_use, time_use) = run_sts_benchmark(sts_data)
scores = df_use

pearson_correlation = scipy.stats.pearsonr(scores, dev_scores) #scores= predicted cos_sim, dev_scores = Overall
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))

Pearson correlation coefficient = -0.6890446935468493
p-value = 1.9816840771934977e-34


In [21]:
df_use = pd.DataFrame(df_use.numpy()).rename(columns={0: "USE"})

In [22]:
df_results = df_results.join(df_use)

In [23]:
df_results.corr()

,Overall,bert_all-MiniLM-L12-v20,USE
Overall,1.000000,-0.719582,-0.689045
bert_all-MiniLM-L12-v20,-0.719582,1.000000,0.879100
USE,-0.689045,0.879100,1.000000


In [24]:
time_use = pd.DataFrame({"time_USE": [time_use,0]})
#pd.DataFrame(index={time_use: 0})

In [25]:
df_use.to_csv('./resultados/USE.csv', index = False)
time_use.to_csv('./resultados/USE_time.csv', index = False)

In [26]:
len(scores)

235

## tf-idf

In [27]:
#df_results = df_pp_test[["pair_id", "Overall"]]
#df_results.head(1)

In [28]:
'''pre_processing_list = [
    {"tokenization": True},
    {"no_url": True, "basic_processing": True, "tokenization": True}
    ]'''


df_pp = pp.pre_process_all(df_text, pre_processing_list)
df_pp_test = pp.pre_process_all(df_text_test, pre_processing_list)

(df_tf_idf, time_tf_idf) = nlp.apply_tf_idf(df_pp_test, len(pre_processing_list))

In [29]:
df_tf_idf.to_csv('./resultados/tf_idf.csv', index = False)
time_tf_idf.to_csv('./resultados/tf_idf_time.csv', index = False)

In [30]:
df_results = df_results.join(df_tf_idf)

In [31]:
df_results.corr()

,Overall,bert_all-MiniLM-L12-v20,USE,tf_idf0
Overall,1.000000,-0.719582,-0.689045,-0.705009
bert_all-MiniLM-L12-v20,-0.719582,1.000000,0.879100,0.862534
USE,-0.689045,0.879100,1.000000,0.943276
tf_idf0,-0.705009,0.862534,0.943276,1.000000


## Salvando resultados da correlação

In [32]:
df_results.to_csv("resultados/all_results.csv")

In [33]:
corr = df_results.corr()
corr

,Overall,bert_all-MiniLM-L12-v20,USE,tf_idf0
Overall,1.000000,-0.719582,-0.689045,-0.705009
bert_all-MiniLM-L12-v20,-0.719582,1.000000,0.879100,0.862534
USE,-0.689045,0.879100,1.000000,0.943276
tf_idf0,-0.705009,0.862534,0.943276,1.000000


In [34]:
corr.to_csv("resultados/pearson_corr.csv")